# UE 803: Data Science
Project: Clustering and Classifying People based on Text and
KB information

About this project
In this project, you will collect information about a people belonging to different categories (such as artists, politician, sportspeople, etc.). Based on this information, you will then try to automatically cluster and classify these people into the correct categories.
We will use two sources of information, namely:

• The Wikipedia online encyclopedia1

• The Wikidata knowledge base2

Deadline

The deadline for submission is May 15th, 2021. This is a strict deadline. Late submissions will be penalised (-0.20 points per day past the deadline).

# Exercise 1 – Corpus extraction (10 points)

The goal of this exercise is to compile a text corpus from the Wikipedia online encyclopedia. This corpus will be made of plain text sentences, selected so as to have a roughly balanced corpus in terms of train-
ing data (each target category should be associated with the same number of sentences). We will focus on the 6 following categories: architects, mathematicians, painters, politicians, singers and writers.
    
Concretely, you are required to implement a Python program which takes as a an input:

• a number k of persons per category,

• a number n of sentences per person (persons whose wikipedia description is too short to have n sentences, should be ignored).

This extraction task can be realised through the following steps:
    
1.(lecture 6) Create a list of persons you want to work with. Those persons should fall into two main types: artists (A) and non artists (Z). Singers, writers and painters are of type A while architects, politi- cians and mathematicians of type Z. For each category, select 30 persons of that category. So in total you should have a list of 180 persons (half of them are artists and half of them are not).

    You can use the wikidata warehouse to find persons of the ex- pected categories. More precisely, the Wikidata collection can be filtered out using the SPARQL language and the following end- point: https://query.wikidata.org/.

    You can thus use the SPARQLwrapper python library to apply a SPARQL query to the Wikidata warehouse and retrieve the re- quired item identifiers.

In [2]:
!pip install SPARQLWrapper

msgpack-numpy 0.4.7.1 requires msgpack>=0.5.2, which is not installed.
thinc 6.10.3 requires msgpack<1.0.0,>=0.5.6, which is not installed.
spacy 2.0.12 has requirement regex==2017.4.5, but you'll have regex 2018.2.21 which is incompatible.
You are using pip version 10.0.1, however version 20.3.4 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
#!pip install --upgrade pip

ERROR: To modify pip, please run the following command:
C:\ProgramData\Anaconda3\envs\tal\python.exe -m pip install --upgrade pip
You are using pip version 10.0.1, however version 20.3.4 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


- Mapping the occupation to ID

This Part is Useless, I just map the ID to occupation. Not affect other block code.

In [3]:

# architects (Q42973)
# mathematician (Q170790)
# painter (Q1028181)
# politician (Q82955)
# singer (Q177220)
# writer (Q36180)


In [4]:
k = 30 # 30 people
n = 20 # 20 sentences
a = ['architects', 'mathematicians', 'politicians', 'singers', 'writers','painters']
# A = []
a1 = ['Q42973','Q170790','Q82955','Q177220','Q36180','Q1028181']

In [5]:
persons = dict(zip(a,a1))
A = ['singers', 'writers','painters']
A1= ['Q177220','Q36180','Q1028181']
persons_A = dict(zip(A,A1))
Z = ['architects', 'mathematicians', 'politicians']
Z1= ['Q42973','Q170790','Q82955']
persons_Z = dict(zip(Z,Z1))

In [6]:
# from pprint import pprint

# pprint(persons)
# pprint(persons_A)
# pprint(persons_Z)


Extract all the data (180 persons and 30 in each categories)


Here is the concept that I extract the data:

SELECT person and personLabel WHERE

{
    
    TYPE A
    
    UNION 
    
    TYPE Z
}

If we go into detail, I extract 30 persons from each categories, so each type = $30 * 3$ (becuase 3 categories in each type). Total = $(30*3)*2$(2 refers types) = 180 persons

The concept of whole script:

SELECT person and personLabel WHERE

{
    
    {
        SELECT person and personLabel WHERE
        {
            {singers}
            
            UNION 
            
            {writers}
            
            UNION 
            
            {painters}
        }
    }
    
    UNION 
    
    {
        SELECT person and personLabel WHERE
        {
            {architects}
            
            UNION 
            
            {mathematicians}
            
            UNION 
            
            {politicians}
        }
    }
}

In [1]:
#-----Run This
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/
#------extract 180 data, I queried all in one
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?person ?personLabel WHERE
{
# type A
  {
    SELECT ?person ?personLabel WHERE
    {
         #singers
        {SELECT ?person ?personLabel
        WHERE
        {
        # ?person  occupation 'singers'
        ?person wdt:P106 wd:Q177220.
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        }
        limit 30
    }
    
    UNION
    
    {
        #writers
        SELECT ?person ?personLabel
        WHERE
        {
        # ?person  occupation writers
        ?person wdt:P106 wd:Q36180.
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        }
        limit 30
    }
  
    UNION
  
    {
        #painters
        SELECT ?person ?personLabel
        WHERE
        {
        #?person  occupation painters
        ?person wdt:P106 wd:Q1028181.
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
         }
        limit 30
        }
    }
  }
  UNION 
  
# type Z
  {
    SELECT ?person ?personLabel WHERE
    {
         #architects
        {SELECT ?person ?personLabel
        WHERE
        {
        # ?person  occupation 'architects'
        ?person wdt:P106 wd:Q42973.
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        }
        limit 30
    }
    
    UNION
    
    {
        #mathematicians
        SELECT ?person ?personLabel
        WHERE
        {
        # ?person  occupation mathematicians
        ?person wdt:P106 wd:Q170790.
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        }
        limit 30
    }
  
    UNION
  
    {
        #politicians
        SELECT ?person ?personLabel
        WHERE
        {
        #?person  occupation politicians
        ?person wdt:P106 wd:Q82955.
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
         }
        limit 30
        }
    }
  }
  
}


"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()



results = get_results(endpoint_url, query)
# lis = []
for result in results["results"]["bindings"]:
    print(result)
#     lis.extend(result)
# len(lis)/2

{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2643'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'George Harrison'}}
{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2632'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Ringo Starr'}}
{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2599'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Paul McCartney'}}
{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2274'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Chet Baker'}}
{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1804'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Barbara'}}
{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1802'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Daniel Balavoine'}}
{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/en

In [2]:
#-----Run This
# -----get the links of each persons i.e: 'http://www.wikidata.org/entity/Q2680'
list_of_wikibases = []
for result in results["results"]["bindings"]:
    list_of_wikibases.append(result['person']['value'])
    
# len(list_of_wikibases)
list_of_wikibases

['http://www.wikidata.org/entity/Q2643',
 'http://www.wikidata.org/entity/Q2632',
 'http://www.wikidata.org/entity/Q2599',
 'http://www.wikidata.org/entity/Q2274',
 'http://www.wikidata.org/entity/Q1804',
 'http://www.wikidata.org/entity/Q1802',
 'http://www.wikidata.org/entity/Q1790',
 'http://www.wikidata.org/entity/Q1779',
 'http://www.wikidata.org/entity/Q1768',
 'http://www.wikidata.org/entity/Q1744',
 'http://www.wikidata.org/entity/Q1716',
 'http://www.wikidata.org/entity/Q1698',
 'http://www.wikidata.org/entity/Q1666',
 'http://www.wikidata.org/entity/Q1631',
 'http://www.wikidata.org/entity/Q1545',
 'http://www.wikidata.org/entity/Q1514',
 'http://www.wikidata.org/entity/Q1453',
 'http://www.wikidata.org/entity/Q1276',
 'http://www.wikidata.org/entity/Q1225',
 'http://www.wikidata.org/entity/Q1203',
 'http://www.wikidata.org/entity/Q1023',
 'http://www.wikidata.org/entity/Q714',
 'http://www.wikidata.org/entity/Q636',
 'http://www.wikidata.org/entity/Q633',
 'http://www.wikida

In [3]:
#-----Run This
#-----get only the wikibase for search
import re


wikibase_list = []

for l in list_of_wikibases:
    x = re.findall("[^http://www.wikidata.org/entity/].*", l) # get the id after "http://www.wikidata.org/entity/"
    wikibase_list.extend(x)
#     print(x)
len(wikibase_list)
wikibase_list

['Q2643',
 'Q2632',
 'Q2599',
 'Q2274',
 'Q1804',
 'Q1802',
 'Q1790',
 'Q1779',
 'Q1768',
 'Q1744',
 'Q1716',
 'Q1698',
 'Q1666',
 'Q1631',
 'Q1545',
 'Q1514',
 'Q1453',
 'Q1276',
 'Q1225',
 'Q1203',
 'Q1023',
 'Q714',
 'Q636',
 'Q633',
 'Q563',
 'Q557',
 'Q512',
 'Q409',
 'Q392',
 'Q303',
 'Q905',
 'Q892',
 'Q862',
 'Q835',
 'Q767',
 'Q765',
 'Q755',
 'Q747',
 'Q742',
 'Q651',
 'Q607',
 'Q605',
 'Q590',
 'Q563',
 'Q557',
 'Q539',
 'Q535',
 'Q512',
 'Q504',
 'Q502',
 'Q501',
 'Q493',
 'Q489',
 'Q448',
 'Q410',
 'Q392',
 'Q377',
 'Q360',
 'Q272',
 'Q42',
 'Q5383',
 'Q5233',
 'Q5074',
 'Q4807',
 'Q4805',
 'Q4768',
 'Q4742',
 'Q4724',
 'Q4538',
 'Q4247',
 'Q4025',
 'Q3824',
 'Q2677',
 'Q2632',
 'Q2599',
 'Q2071',
 'Q1698',
 'Q1500',
 'Q1470',
 'Q1217',
 'Q1203',
 'Q930',
 'Q838',
 'Q765',
 'Q762',
 'Q392',
 'Q352',
 'Q301',
 'Q297',
 'Q296',
 'Q8814',
 'Q8772',
 'Q8763',
 'Q8755',
 'Q8747',
 'Q8743',
 'Q8739',
 'Q8556',
 'Q8287',
 'Q8011',
 'Q7604',
 'Q7286',
 'Q7259',
 'Q7103',
 'Q7099',

2.(lecture 3) for each selected person, retrieve his.her Wikidata description and Wikipedia page title. This can be done using the wikidata API along with the wptools python library.

In [92]:

# #test error API i.e: 'Q1503'
# import wptools
# lis_test = ['Q2680',
#             'Q1503',
#             'Q2643', 
#             'Q2632']
# for l in lis_test:
#     try:
#         page = wptools.page(wikibase=l)
#         page.get_wikidata()
#         page.data['description']
#     except LookupError  as le:
#         pass

In [4]:
#-----Run This

import wptools
# lis_test = ['Q2680',
#             'Q2643', 
#             'Q2632']
# for wikibase in list_of_wikibases:
# wikibase= 'Q1803'

page_descriptions = []
add_descriptions_list = []
page_titles= []
add_titles_list = []
for wikibase in wikibase_list:
    try: 
        page = wptools.page(wikibase=wikibase)
        page.get_wikidata()
        #-----collect list of descriptions
        page_descriptions.append(page.data['description'])
        page_descriptions_copy = page_descriptions.copy()
        add_descriptions_list.append(page_descriptions_copy)
        page_descriptions[100:]

        #-----collect list of titles
    #     page.data['title']
        page_titles.append(page.data['title']) # add one title name to page_titles list
        page_titles_copy = page_titles.copy() # copy the page_titles list
        add_titles_list.append(page_titles_copy) # add selected title to another list called add_titles_list 
        page_titles[100:] # truncate all the data in page_titles list
    
    except LookupError  as le:
        pass



www.wikidata.org (wikidata) Q2643
www.wikidata.org (labels) P4785|Q15242531|P3106|Q61285|P40|P106|P...
www.wikidata.org (labels) P2600|P2949|Q17172850|P1207|P6870|Q7120...
www.wikidata.org (labels) Q3140857|P3017|P4781|P8814|Q8488188|P20...
www.wikidata.org (labels) P1368|P19|P409|P3909|P7902|Q15921732|P5...
www.wikidata.org (labels) P213|P3040|Q22344608|P27|Q208909|Q14150...
www.wikidata.org (labels) P5617|P1266|P1220|P2002|P7666|P509|Q497...
en.wikipedia.org (imageinfo) File:George Harrison 1974.jpg
George Harrison (en) data
{
  aliases: <list(3)> The Quiet Beatle, Nelson Wilbury, 00013482508 IPI
  claims: <dict(200)> P40, P19, P20, P106, P26, P166, P358, P373, ...
  description: <str(62)> British musician and lead guitarist of th...
  image: <list(1)> {'file': 'File:George Harrison 1974.jpg', 'kind...
  label: George Harrison
  labels: <dict(289)> P4785, Q15242531, P3106, Q61285, P40, P106, ...
  modified: <dict(1)> wikidata
  requests: <list(8)> wikidata, labels, labels, labels, la

www.wikidata.org (labels) P3106|P2639|P5719|P40|P1477|P6911|P106|...
www.wikidata.org (labels) P4208|Q55550590|P1695|Q193023|Q335017|Q...
www.wikidata.org (labels) P373|P1050|Q1124849|P1559|Q98608837|P31...
www.wikidata.org (labels) P269|P1741|P7859|P6578|P1670|P5361|Q673...
www.wikidata.org (labels) Q1196752|P6683|Q10800557|P2722|P101|Q15...
www.wikidata.org (labels) P5121|P949|P227|P4104|P1284|P4491|P3417
en.wikipedia.org (imageinfo) File:Ella Fitzgerald 1962.JPG
Ella Fitzgerald (en) data
{
  aliases: <list(4)> First Lady of Song, Queen of Jazz, Lady Ella,...
  claims: <dict(205)> P434, P119, P358, P373, P214, P213, P19, P20...
  description: American jazz singer
  image: <list(1)> {'file': 'File:Ella Fitzgerald 1962.JPG', 'kind...
  label: Ella Fitzgerald
  labels: <dict(257)> P3106, P2639, P5719, P40, P1477, P6911, P106...
  modified: <dict(1)> wikidata
  requests: <list(8)> wikidata, labels, labels, labels, labels, la...
  title: Ella_Fitzgerald
  what: human
  wikibase: Q1768
  w

www.wikidata.org (labels) P6903|P3569|Q183387|P2013|P6722|P8849|P...
www.wikidata.org (labels) P1902|P109|P345|Q30|P2163|P19|Q935843|P...
www.wikidata.org (labels) P8349|Q12152|Q2995893|P5587|P1286|P3348...
en.wikipedia.org (imageinfo) File:Janis Joplin in 1969 (cropped)....
Janis Joplin (en) data
{
  aliases: <list(2)> Janis Lyn Joplin, Pearl
  claims: <dict(145)> P1315, P214, P213, P106, P434, P373, P244, P...
  description: American singer and songwriter
  image: <list(2)> {'file': 'File:Janis Joplin in 1969 (cropped).p...
  label: Janis Joplin
  labels: <dict(191)> P3106, P2639, P463, P1477, P106, P5019, P200...
  modified: <dict(1)> wikidata
  requests: <list(6)> wikidata, labels, labels, labels, labels, im...
  title: Janis_Joplin
  what: human
  wikibase: Q1514
  wikidata: <dict(145)> NLA Trove ID (P1315), VIAF ID (P214), ISNI...
  wikidata_pageid: 1969
  wikidata_url: https://www.wikidata.org/wiki/Q1514
}
www.wikidata.org (wikidata) Q1453
www.wikidata.org (labels) P106|P166|P85

www.wikidata.org (labels) Q639669|P3630|P5534|P7763|P3569|P2013|Q...
www.wikidata.org (labels) P1280|Q5593754|P1412|P1902|P3373|Q54370...
www.wikidata.org (labels) Q843402|P569|P69|Q16|P349|P910|P2604|P2...
www.wikidata.org (labels) P7293|P244|Q1542199|Q12195|P3478|Q66706...
www.wikidata.org (labels) P5121|P949|P227|P4104|Q6314105|P3417
en.wikipedia.org (imageinfo) File:Neil Young - Per Ole Hagen.jpg|...
Neil Young (en) data
{
  aliases: <list(4)> Neil Percival Young, Shakey, Godfather of Gru...
  claims: <dict(149)> P735, P22, P106, P214, P434, P373, P358, P27...
  description: Canadian-American guitarist, singer and songwriter
  image: <list(2)> {'file': 'File:Neil Young - Per Ole Hagen.jpg',...
  label: Neil Young
  labels: <dict(256)> P3106, P2639, P463, P1477, P106, Q38447, P50...
  modified: <dict(1)> wikidata
  requests: <list(8)> wikidata, labels, labels, labels, labels, la...
  title: Neil_Young
  what: human
  wikibase: Q633
  wikidata: <dict(149)> given name (P735), father (

en.wikipedia.org (imageinfo) File:Kafka1906 cropped.jpg|File:Kafka.jpg
Franz Kafka (en) data
{
  aliases: <list(2)> František Kafka, Kafka
  claims: <dict(258)> P1233, P910, P119, P106, P108, P214, P227, P...
  description: Bohemian novelist and short story writer
  image: <list(2)> {'file': 'File:Kafka1906 cropped.jpg', 'kind': ...
  label: Franz Kafka
  labels: <dict(333)> P9081, P1233, P40, P5019, P1890, P4293, P650...
  modified: <dict(1)> wikidata
  requests: <list(9)> wikidata, labels, labels, labels, labels, la...
  title: Franz_Kafka
  what: human
  wikibase: Q905
  wikidata: <dict(257)> Internet Speculative Fiction Database auth...
  wikidata_pageid: 1228
  wikidata_url: https://www.wikidata.org/wiki/Q905
}
www.wikidata.org (wikidata) Q892
www.wikidata.org (labels) P40|P1233|P5019|Q222595|P410|Q361|P1890...
www.wikidata.org (labels) Q2532287|Q81834|P1411|P19|P6394|P103|Q8...
www.wikidata.org (labels) P2734|Q10295543|P856|P3782|P535|P1953|P...
www.wikidata.org (labels) P3987|P2

...

www.wikidata.org (labels) P106|P6735|P5019|P4177|Q37068|P650|P166...
www.wikidata.org (labels) Q2584571|P8189|P8950|P1472|P3791|P2843|...
www.wikidata.org (labels) P6379|P569|P69|P910|Q18084456|P20|P213|...
en.wikipedia.org (imageinfo) File:Brjullov.jpg
Karl Bryullov (en) data
{
  aliases: <list(31)> Karl Briullov, Carlo Brulleau, Carlo Brullo,...
  claims: <dict(100)> P691, P119, P373, P214, P213, P19, P20, P227...
  description: Russian painter (1799-1852)
  image: <list(1)> {'file': 'File:Brjullov.jpg', 'kind': 'wikidata...
  label: Karl Bryullov
  labels: <dict(144)> P106, P6735, P5019, P4177, Q37068, P650, P16...
  modified: <dict(1)> wikidata
  requests: <list(5)> wikidata, labels, labels, labels, imageinfo
  title: Karl_Bryullov
  what: human
  wikibase: Q4768
  wikidata: <dict(99)> NKCR AUT ID (P691), place of burial (P119),...
  wikidata_pageid: 5719
  wikidata_url: https://www.wikidata.org/wiki/Q4768
}
www.wikidata.org (wikidata) Q4724
www.wikidata.org (labels) P463|P2639|P31

www.wikidata.org (labels) P109|Q298|P106|Q18882653|P2163|Q5576697...
www.wikidata.org (labels) P935|Q40348|P20|P213|Q6581097|Q55331661...
en.wikipedia.org (imageinfo) File:Gabriel Gonzalez Videla.jpg|Fil...
Gabriel González Videla (en) data
{
  aliases: <list(3)> Gabriel González Videla, Gabriel Gonzales Vid...
  claims: <dict(45)> P214, P213, P109, P18, P227, P268, P269, P244...
  description: Chilean politician (1898-1980)
  image: <list(2)> {'file': 'File:Gabriel Gonzalez Videla.jpg', 'k...
  label: Gabriel Gonzáles Videla
  labels: <dict(66)> P109, Q298, P106, Q18882653, P2163, Q5576697,...
  modified: <dict(1)> wikidata
  requests: <list(4)> wikidata, labels, labels, imageinfo
  title: Gabriel_González_Videla
  what: human
  wikibase: Q815
  wikidata: <dict(45)> VIAF ID (P214), ISNI (P213), signature (P10...
  wikidata_pageid: 1133
  wikidata_url: https://www.wikidata.org/wiki/Q815
}
www.wikidata.org (wikidata) Q651
www.wikidata.org (labels) P463|P2799|P106|P5019|Q188|Q214917|Q622

www.wikidata.org (labels) P3782|P6412|P535|P1953|P6058|Q7367470|P...
www.wikidata.org (labels) P349|P26|Q12192|P20|P947|P27|P800|P1296...
www.wikidata.org (labels) Q602358|P1576|P2949|Q3271923|P1207|P172...
www.wikidata.org (labels) P5641|P2889|Q210115|P9164|Q30738521|P73...
www.wikidata.org (labels) Q1569226|P3630|P3965|P570|Q3271708|P121...
www.wikidata.org (labels) Q90|Q3059936|P4200|P1649|P8094|P5882|P1...
en.wikipedia.org (imageinfo) File:Victor Hugo by Étienne Carjat 1...
Victor Hugo (en) data
{
  aliases: <list(3)> Victor Marie Hugo, Victor-Marie Hugo, Victor ...
  claims: <dict(292)> P1045, P18, P109, P106, P27, P40, P22, P25, ...
  description: French poet, novelist, and dramatist (1802-1885)
  image: <list(1)> {'file': 'File:Victor Hugo by Étienne Carjat 18...
  label: Victor Hugo
  labels: <dict(385)> P40, P1233, P5019, P1890, P650, P9251, Q3592...
  modified: <dict(1)> wikidata
  requests: <list(10)> wikidata, labels, labels, labels, labels, l...
  title: Victor_Hugo
  what

www.wikidata.org (labels) P7928|P9081|P40|Q1621139|P5019|P410|Q31...
www.wikidata.org (labels) P3154|P8814|P1559|Q306626|Q207826|P646|...
www.wikidata.org (labels) Q154554|P2435|Q7824282|Q42884|P5002|P12...
www.wikidata.org (labels) P2019|Q677676|Q52307175|P2188|P8992|P31...
www.wikidata.org (labels) P8349|P4084|Q158314|P9179|Q15958642|Q15...
www.wikidata.org (labels) P271|P8849|Q22084744|Q76106|P8568|Q2763...
www.wikidata.org (labels) P39|P6683|P8851|P25|P648|P1889|Q2140674...
www.wikidata.org (labels) Q4613831|P6404|P1344|Q18125641|P140|Q41...
www.wikidata.org (labels) P7822|Q461|P3847|P1368|Q7310|Q20078554|...
www.wikidata.org (labels) P949
en.wikipedia.org (imageinfo) File:Hitler portrait crop.jpg
Adolf Hitler (en) data
{
  aliases: <list(4)> Führer, Bohemian Corporal, A. H., Der Führer
  claims: <dict(288)> P1576, P1150, P27, P19, P20, P26, P22, P106,...
  description: <str(103)> Austrian-born German politician, chancel...
  image: <list(1)> {'file': 'File:Hitler portrait crop.jpg

www.wikidata.org (labels) P1477|Q839078|P106|P5019|P2003|Q194540|...
www.wikidata.org (labels) P7822|P8951|P19|P409|Q1680589|Q8817796|...
www.wikidata.org (labels) Q486748|Q1631137|P735|P1263|P935|P3219|...
en.wikipedia.org (imageinfo) File:Stephen-Harper-Cropped-2014-02-...
Stephen Harper (en) data
{
  aliases: <list(1)> Stephen Joseph Harper
  claims: <dict(72)> P735, P31, P27, P26, P19, P140, P214, P213, P...
  description: Canadian politician
  image: <list(2)> {'file': 'File:Stephen Harper by Remy Steinegge...
  label: Stephen Harper
  labels: <dict(111)> P1477, Q839078, P106, P5019, P2003, Q194540,...
  modified: <dict(1)> wikidata
  requests: <list(5)> wikidata, labels, labels, labels, imageinfo
  title: Stephen_Harper
  what: human
  wikibase: Q206
  wikidata: <dict(72)> given name (P735), instance of (P31), count...
  wikidata_pageid: 348
  wikidata_url: https://www.wikidata.org/wiki/Q206
}
www.wikidata.org (wikidata) Q192
www.wikidata.org (labels) P3106|P463|P3051|P1477|P40|P

In [6]:
#-----Run This

import pandas as pd
from pprint import pprint
# print(len(page_titles))
# print(len(page_descriptions))
# pprint(page_titles)
# pprint(page_descriptions)
des_list = []
column_des = ['Descriptions']
for des in page_descriptions:
    des_list.append(des)
    #         text_in.append(text)
    #         df = pd.DataFrame(text_in,columns = columns)
    df_des = pd.DataFrame(des_list,columns = column_des) #import text to dataFrame
    
    des_list[100:]
#     title_in_df[100:]

In [7]:
df_des

,Descriptions
0,British musician and lead guitarist of the Bea...
1,"British musician, drummer for the Beatles"
2,"English singer-songwriter, bassist of The Beatles"
3,American jazz trumpeter and vocalist (1929-1988)
4,French recording artist; singer-songwriter
...,...
175,Canadian politician
176,former prime minister of the United Kingdom (b...
177,16th president of the United States (1809-1865)
178,44th president of the United States


3.(lecture 3) Once you have a list of wikipedia page titles, fetch (if it exists) the corresponding English wikipedia page, and extract the n first sentences of its content.

At the end of this process, you will have in memory, for each person you selected:
    
• the text of the corresponding Wikipedia page ;

• the corresponding data type (A or Z) and category ;

• the WikiData description.

Store these into a csv or a json file and save it on your hard drive. NB: Note there is some overlapping with lab session 1.

In [1]:
# import wikipedia 
# wikipedia.set_lang("en")
# # page2 = wikipedia.page('Neil_Young') 
# # content = page2.content
# summary = wikipedia.summary('Neil_Young', sentences = 10)
# summary
# # content

In [59]:
# import pandas as pd

# # data_in = ['95 album Mirror Ball with Pearl Jam. More recently he has been backed by Promise of the Real.']
# data = [['NaN']]

# # data.append(data_in)

# columns = ['Text']
# df = pd.DataFrame(data,columns = columns)
# df


,Text
0,NaN


In [2]:
# !pip install wikipedia

In [11]:
#-----Run This

import wikipedia
from pprint import pprint
import pandas as pd

# pprint(page_titles)
# pprint(page_descriptions)
# list_of_contents = []
# text_in = []
text = []
title_in_df = []
column_text = ['Text']
column_title = ['title']
for title in page_titles:
    try:
        wikipedia.set_lang("en")
        page2 = wikipedia.page(title) 
        text.append(page2.content)
        title_in_df.append(title)
#         text_in.append(text)
#         df = pd.DataFrame(text_in,columns = columns)
        df_text = pd.DataFrame(text,columns = column_text) #import text to dataFrame
        df_title = pd.DataFrame(title_in_df,columns = column_title)
        text[100:]
        title_in_df[100:]

    except AttributeError as ae:
#         print(ae)
        pass
    except wikipedia.exceptions.PageError as pe:
#         print(pe)
        pass
    except wikipedia.exceptions.DisambiguationError as de:
        print(de)
        pass



/Users/neath/opt/anaconda3/envs/ds/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/neath/opt/anaconda3/envs/ds/lib/python3.8/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


"gotti" may refer to: 
Gotti (surname)
Juan Gotti
Boo & Gotti
Yo Gotti
Kurupt
Gotti (1996 film)
Gotti (2018 film)
"Gotti" (song)
Getting Gotti
Growing Up Gotti
"rafael" may refer to: 
Rafael (given name)
Rafael, California
Rafael Advanced Defense Systems
Hurricane Rafael
Rafael (telenovela)
Rafaël (film)
Rafael (footballer, born 1982)
Rafael (footballer, born 1987)
Rafael (footballer, born 1990)
Alex Rafael
Raphael (disambiguation)
Dassault Rafale
San Rafael (disambiguation)
Raffaele
"George Bush" may refer to: 
George H. W. Bush
Presidency of George H. W. Bush
George W. Bush
Presidency of George W. Bush
George Bush (biblical scholar)
George Bush (footballer)
George Bush (racing driver)
George H. Bush
George P. Bush
George Washington Bush
USS George H.W. Bush
All pages with titles containing George Bush
All pages with titles containing George H. W. Bush
All pages with titles containing George W. Bush
Bush family
Presidency of George Bush (disambiguation)
George Brush (disambiguation)
B

In [12]:
#-----Run This
#-----Above didnt work properly, so need to run this block
#-----Note: need to run this above code; otherwise, it will crash lol


import wikipedia
import pandas as pd

text = []
title_in_df = []
column_text = ['Text']
column_title = ['title']



# Non=[['NaN']]

for title in page_titles:
#     page2 = wikipedia.page(title) 
    try:
        wikipedia.set_lang("en")
        page2 = wikipedia.page(title) 
        text.append(page2.content)
        title_in_df.append(title)
    #         text_in.append(text)
    #         df = pd.DataFrame(text_in,columns = columns)
        df_text = pd.DataFrame(text,columns = column_text) #import text to dataFrame
        df_title = pd.DataFrame(title_in_df,columns = column_title)
        text[100:]
        title_in_df[100:]
        
    except AttributeError as ae:
#         print(ae)
        text.append(page2.content)
        title_in_df.append(title)
        df_text = pd.DataFrame(text,columns = column_text) #import text to dataFrame
        df_title = pd.DataFrame(title_in_df,columns = column_title)
        text[100:]
  
        title_in_df[100:]
    except wikipedia.exceptions.PageError as pe:
        text.append(page2.content)
        title_in_df.append(title)
        df_text = pd.DataFrame(text,columns = column_text) #import text to dataFrame
        df_title = pd.DataFrame(title_in_df,columns = column_title)
        text[100:]

        title_in_df[100:]
#         print(pe)
    #         pass
    except wikipedia.exceptions.DisambiguationError as de:
        text.append(page2.content)
        title_in_df.append(title)
        df_text = pd.DataFrame(text,columns = column_text) #import text to dataFrame
        df_title = pd.DataFrame(title_in_df,columns = column_title)
        text[100:]
     
        title_in_df[100:]
#         print(de)
    #         pass

In [13]:
df_text

,Text
0,"George Washington (February 22, 1732 – Decembe..."
1,"George Washington (February 22, 1732 – Decembe..."
2,"George Washington (February 22, 1732 – Decembe..."
3,"Chesney Henry ""Chet"" Baker Jr. (December 23, 1..."
4,Monique Andrée Serf (9 June 1930 – 24 November...
...,...
175,"Stephen Joseph Harper (born April 30, 1959) i..."
176,David William Donald Cameron (born 9 October ...
177,"Abraham Lincoln (; February 12, 1809 – April 1..."
178,Barack Hussein Obama II ( (listen) bə-RAHK hoo...


In [14]:
df_title

,title
0,George_Harrison
1,Ringo_Starr
2,Paul_McCartney
3,Chet_Baker
4,Barbara_(singer)
...,...
175,Stephen_Harper
176,David_Cameron
177,Abraham_Lincoln
178,Barack_Obama


In [15]:
df = pd.concat((df_title,df_des), axis = 1)
df_final = pd.concat((df,df_text),axis = 1)
df_final

,title,Descriptions,Text
0,George_Harrison,British musician and lead guitarist of the Bea...,"George Washington (February 22, 1732 – Decembe..."
1,Ringo_Starr,"British musician, drummer for the Beatles","George Washington (February 22, 1732 – Decembe..."
2,Paul_McCartney,"English singer-songwriter, bassist of The Beatles","George Washington (February 22, 1732 – Decembe..."
3,Chet_Baker,American jazz trumpeter and vocalist (1929-1988),"Chesney Henry ""Chet"" Baker Jr. (December 23, 1..."
4,Barbara_(singer),French recording artist; singer-songwriter,Monique Andrée Serf (9 June 1930 – 24 November...
...,...,...,...
175,Stephen_Harper,Canadian politician,"Stephen Joseph Harper (born April 30, 1959) i..."
176,David_Cameron,former prime minister of the United Kingdom (b...,David William Donald Cameron (born 9 October ...
177,Abraham_Lincoln,16th president of the United States (1809-1865),"Abraham Lincoln (; February 12, 1809 – April 1..."
178,Barack_Obama,44th president of the United States,Barack Hussein Obama II ( (listen) bə-RAHK hoo...


In [16]:
df_final.to_csv("test_data.csv") # write to csv file 

In [17]:
!pip install openpyxl

     |████████████████████████████████| 243 kB 2.2 MB/s eta 0:00:01
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-py3-none-any.whl size=8913 sha256=f24dbb61016f2b01f5fa1f3829b0714cce215bd14bc249ffc07d3fe9dc3f66ca
  Stored in directory: /Users/neath/Library/Caches/pip/wheels/6e/df/38/abda47b884e3e25f9f9b6430e5ce44c47670758a50c0c51759
Successfully built et-xmlfile


Add Type and categories

In [18]:
df_final.to_excel("test_data.xlsx")

In [49]:
import pandas as pd
df_cate = pd.read_excel("test_data.xlsx")
df_cate

,Unnamed: 0,title,Descriptions,Text
0,0,George_Harrison,British musician and lead guitarist of the Bea...,"George Washington (February 22, 1732 – Decembe..."
1,1,Ringo_Starr,"British musician, drummer for the Beatles","George Washington (February 22, 1732 – Decembe..."
2,2,Paul_McCartney,"English singer-songwriter, bassist of The Beatles","George Washington (February 22, 1732 – Decembe..."
3,3,Chet_Baker,American jazz trumpeter and vocalist (1929-1988),"Chesney Henry ""Chet"" Baker Jr. (December 23, 1..."
4,4,Barbara_(singer),French recording artist; singer-songwriter,Monique Andrée Serf (9 June 1930 – 24 November...
...,...,...,...,...
175,175,Stephen_Harper,Canadian politician,"Stephen Joseph Harper (born April 30, 1959) i..."
176,176,David_Cameron,former prime minister of the United Kingdom (b...,David William Donald Cameron (born 9 October ...
177,177,Abraham_Lincoln,16th president of the United States (1809-1865),"Abraham Lincoln (; February 12, 1809 – April 1..."
178,178,Barack_Obama,44th president of the United States,Barack Hussein Obama II ( (listen) bə-RAHK hoo...


In [51]:
df = pd.DataFrame(columns=('Category', 'type'))
c = ['singers', 'writers','painters','architects', 'mathematicians', 'politicians']

for i in range(180):
    if i<30:
        df.loc[i] = [c[0], 'A']
    elif i < 60:
        df.loc[i] = [c[1], 'A']
    elif i < 90:
        df.loc[i] = [c[2], 'A']
    elif i < 120:
        df.loc[i] = [c[3], 'Z']
    elif i < 150:
        df.loc[i] = [c[4], 'Z']
    else:
        df.loc[i] = [c[5], 'Z']
df

,Category,type
0,singers,A
1,singers,A
2,singers,A
3,singers,A
4,singers,A
...,...,...
175,politicians,Z
176,politicians,Z
177,politicians,Z
178,politicians,Z


In [53]:
new_df = pd.DataFrame({'Person': df_cate['title'], 'Descriptions': df_cate['Descriptions'], 'Categories': df['Category'],'Type': df['type'],'Text': df_cate['Text']})

In [54]:
new_df

,Person,Descriptions,Categories,Type,Text
0,George_Harrison,British musician and lead guitarist of the Bea...,singers,A,"George Washington (February 22, 1732 – Decembe..."
1,Ringo_Starr,"British musician, drummer for the Beatles",singers,A,"George Washington (February 22, 1732 – Decembe..."
2,Paul_McCartney,"English singer-songwriter, bassist of The Beatles",singers,A,"George Washington (February 22, 1732 – Decembe..."
3,Chet_Baker,American jazz trumpeter and vocalist (1929-1988),singers,A,"Chesney Henry ""Chet"" Baker Jr. (December 23, 1..."
4,Barbara_(singer),French recording artist; singer-songwriter,singers,A,Monique Andrée Serf (9 June 1930 – 24 November...
...,...,...,...,...,...
175,Stephen_Harper,Canadian politician,politicians,Z,"Stephen Joseph Harper (born April 30, 1959) i..."
176,David_Cameron,former prime minister of the United Kingdom (b...,politicians,Z,David William Donald Cameron (born 9 October ...
177,Abraham_Lincoln,16th president of the United States (1809-1865),politicians,Z,"Abraham Lincoln (; February 12, 1809 – April 1..."
178,Barack_Obama,44th president of the United States,politicians,Z,Barack Hussein Obama II ( (listen) bə-RAHK hoo...


In [55]:
new_df.to_excel("test_data2.xlsx")

As we already fetch Wikipedia page titles and fetch (if it exists) the corresponding English wikipedia page. Therefore, the final result of the people is 126 (54 was not match).

We filter manually by using excel to get the distinct contents. We didnt filter in the code 

# Exercise 2 – Pre-processing (3 points)

Pre-processing (3 points)

Before applying machine learning algorithm to your data, you first need to process text. For each Wikipedia text collected, do the follow- ing:

• tokenize the text

• lowercase the tokens

• remove function words


In [56]:
import pandas as pd
new_df = pd.read_excel("test_data2_clean.xlsx")
new_df


,Person,Descriptions,Categories,Type,Text
0,George_Harrison,British musician and lead guitarist of the Bea...,singers,A,"George Washington (February 22, 1732 – Decembe..."
1,Chet_Baker,American jazz trumpeter and vocalist (1929-1988),singers,A,"Chesney Henry ""Chet"" Baker Jr. (December 23, 1..."
2,Barbara_(singer),French recording artist; singer-songwriter,singers,A,Monique Andrée Serf (9 June 1930 – 24 November...
3,Daniel_Balavoine,French recording artist; singer-songwriter,singers,A,Daniel Balavoine (5 February 1952 – 14 January...
4,Louis_Armstrong,"American jazz trumpeter, composer and singer",singers,A,"Louis Daniel Armstrong (August 4, 1901 – July ..."
...,...,...,...,...,...
121,Sebastián_Piñera,35th and 37th president of Chile,politicians,Z,Miguel Juan Sebastián Piñera Echenique (Spanis...
122,Stephen_Harper,Canadian politician,politicians,Z,"Stephen Joseph Harper (born April 30, 1959) i..."
123,David_Cameron,former prime minister of the United Kingdom (b...,politicians,Z,David William Donald Cameron (born 9 October ...
124,Abraham_Lincoln,16th president of the United States (1809-1865),politicians,Z,"Abraham Lincoln (; February 12, 1809 – April 1..."


In [57]:
import nltk
# Tokenizing

def preprocessing(text):
    sentences= nltk.sent_tokenize(text) 
    new_text= []
    for row in sentences:
    # tokenize
        tokens = nltk.tokenize.word_tokenize(row)
        # lowercase
        tokens = [token.lower() for token in tokens]
        # isword
        tokens = [token for token in tokens if token.isalpha()]
        clean_sentence = ''
        clean_sentence = ' '.join(token for token in tokens)
        new_text.append(clean_sentence)
#         clean.append(clean_sentence)
        
    return ' '.join(new_text)

preprocessing("John loves 33 apples on day59")

'john loves apples on'

Do the same for the Wikidata description and store the results in a panda dataframe containing the following columns.

• column 1: person

• column 2: Wikipedia page text

• column 3: Wikipedia page text after preprocessing • 

• column 4: Wikidata description

• column 5: Wikidata description after preprocessing


In [58]:

# add new column to store the cleaned text
new_df["clean_text"]= new_df["Text"].apply(preprocessing)
new_df["clean_description"]= new_df["Descriptions"].apply(preprocessing)
new_df

,Person,Descriptions,Categories,Type,Text,clean_text,clean_description
0,George_Harrison,British musician and lead guitarist of the Bea...,singers,A,"George Washington (February 22, 1732 – Decembe...",george washington february december was an ame...,british musician and lead guitarist of the bea...
1,Chet_Baker,American jazz trumpeter and vocalist (1929-1988),singers,A,"Chesney Henry ""Chet"" Baker Jr. (December 23, 1...",chesney henry chet baker december may was an a...,american jazz trumpeter and vocalist
2,Barbara_(singer),French recording artist; singer-songwriter,singers,A,Monique Andrée Serf (9 June 1930 – 24 November...,monique andrée serf june november was a french...,french recording artist
3,Daniel_Balavoine,French recording artist; singer-songwriter,singers,A,Daniel Balavoine (5 February 1952 – 14 January...,daniel balavoine february january was a french...,french recording artist
4,Louis_Armstrong,"American jazz trumpeter, composer and singer",singers,A,"Louis Daniel Armstrong (August 4, 1901 – July ...",louis daniel armstrong august july nicknamed s...,american jazz trumpeter composer and singer
...,...,...,...,...,...,...,...
121,Sebastián_Piñera,35th and 37th president of Chile,politicians,Z,Miguel Juan Sebastián Piñera Echenique (Spanis...,miguel juan sebastián piñera echenique spanish...,and president of chile
122,Stephen_Harper,Canadian politician,politicians,Z,"Stephen Joseph Harper (born April 30, 1959) i...",stephen joseph harper born april is a canadian...,canadian politician
123,David_Cameron,former prime minister of the United Kingdom (b...,politicians,Z,David William Donald Cameron (born 9 October ...,david william donald cameron born october is a...,former prime minister of the united kingdom born
124,Abraham_Lincoln,16th president of the United States (1809-1865),politicians,Z,"Abraham Lincoln (; February 12, 1809 – April 1...",abraham lincoln february april was an american...,president of the united states


Note. To improve clustering and classification results, feel free to add further pre-processing steps (eg Named entity recognition, pos- tagging and extraction of e.g., nouns and verbs).

# Exercise 3 – Clustering (3 points)

The goal of this exercise is (i) to use the collected data (text and wikidata descriptions) to automatically cluster persons first, using
2 clusters and second, using 6 clusters and (ii) to compare the results obtained when using three ways of representing text (tokens, token frequency and tf-idf) and 2 vs. 6 clusters.
Your code should include the following functions:
    
• a function to train a clustering algorithm on some data using N clusters and some input representation method M (tokens, token frequency and tf-idf). Data, M and N should be parameters of that function.

• a function to compute both intrinsic (Silhouette coefficient) and extrinsic (homogeneity, completeness, v-measure, adjusted Rand index) evaluation scores for clustering results.

• a function to visualise those metrics values for each of the three input representations (tokens, token frequency and tf-id) and for 2 vs. 6 clusters (so your visualisation should display 5 scores for each of the 6 clustering results).

In [ ]:
# N clusters
# M method (tokens, token frequency and tf-idf)
def clustering(data,method,num_clus):

    
def clustering_evaluation():
    
def clustering visualization():
    
    

# Exercise 4 – Classifying (4 points)

Since you know which category and subcategory each person in your dataset belongs to, you can also learn a classifier and check how well
it can predict the category and subcategory a person in your dataset belongs to.

    Your code should include:

• a function which outputs accuracy, a confusion matrix, precision, recall and F1 for the results of your classification (when classifying into categories and when classifying into subcategories)

• a function which outputs a visualisation of the accuracy of your classifier per category and per subcategories